In [ ]:
#解壓縮檔案
from google.colab import drive
drive.mount("/content/drive")
import os
path="/content/drive/MyDrive"
os.chdir(path)
os.listdir(path)
!pwd 1
!ls
!unzip '/content/drive/MyDrive/CVFinalproject/archive.zip' -d '/content/drive/MyDrive/CarLicensePlateDetection'

In [ ]:
import numpy as np
import pandas as pd
import os
import numpy

In [ ]:
os.mkdir("/content/plate_train_data")
os.mkdir("/content/plate_train_data/images")
os.mkdir("/content/plate_train_data/labels")
os.mkdir("/content/plate_train_data/images/train")
os.mkdir("/content/plate_train_data/images/val")
os.mkdir("/content/plate_train_data/labels/train")
os.mkdir("/content/plate_train_data/labels/val")

In [ ]:
#轉換為YOLO所使用的.txt格式
import xml.etree.ElementTree as ET
from xml.dom.minidom import parse

path = "/content/drive/MyDrive/CarLicensePlateDetection/annotations"
classes = {"licence":0}

for annotations in os.listdir(path):
    dom = parse(os.path.join(path,annotations))
    root = dom.documentElement
    filename = ".txt".join(root.getElementsByTagName("filename")[0].childNodes[0].data.split(".png"))
    image_width = root.getElementsByTagName("width")[0].childNodes[0].data
    image_height = root.getElementsByTagName("height")[0].childNodes[0].data
    with open("/content/plate_train_data/labels/train/"+filename,"w") as r:
        for items in root.getElementsByTagName("object") :
            name = items.getElementsByTagName("name")[0].childNodes[0].data
            xmin = items.getElementsByTagName("xmin")[0].childNodes[0].data
            ymin = items.getElementsByTagName("ymin")[0].childNodes[0].data
            xmax = items.getElementsByTagName("xmax")[0].childNodes[0].data
            ymax = items.getElementsByTagName("ymax")[0].childNodes[0].data
            x_center_norm = ((int(xmin)+int(xmax)) / 2 ) / int(image_width)
            y_center_norm = ((int(ymin)+int(ymax))/2) / int(image_height)
            width_norm = ((int(xmax)-int(xmin))/int(image_width))
            height_norm = ((int(ymax)-int(ymin))/int(image_height))
            r.write(str(classes[name])+" ")
            r.write(str(x_center_norm)+" ")
            r.write(str(y_center_norm)+" ")
            r.write(str(width_norm)+" ")
            r.write(str(height_norm)+"\n")

In [ ]:
import shutil
path = "/content/drive/MyDrive/CarLicensePlateDetection/images"

for images in os.listdir(path):
  image_path = os.path.join(path,images)
  shutil.copy(image_path,"/content/plate_train_data/images/train/"+images)

In [ ]:
from random import shuffle
from glob import glob
import random
import shutil

random.seed(42)
files = glob("/content/plate_train_data/images/train/*.png")
shuffle(files)
txt_train_path = "/content/plate_train_data/labels/train/"
img_train_path = "/content/plate_train_data/images/train/"
txt_val_path = "/content/plate_train_data/labels/val/"
img_val_path = "/content/plate_train_data/images/val/"
for i in files[:120]:
    filename = i.split("/")[-1]
    shutil.move(txt_train_path+filename.replace(".png",".txt"),\
                txt_val_path+filename.replace(".png",".txt"))
    shutil.move(img_train_path+filename,img_val_path+filename)

In [ ]:
%cd /content
!echo "train: /content/plate_train_data/images/train" > licence.yaml
!echo "val:   /content/plate_train_data/images/val"  >> licence.yaml
!echo "nc : 1" >> licence.yaml
!echo "names: ['licence']" >> licence.yaml

In [ ]:
%cd /content/
%rm -rf yolov5
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

In [ ]:
!python train.py --weights yolov5s.pt --cfg models/yolov5s.yaml --img 416 --batch 48 --epochs 40 --data /content/licence.yaml  --cache

In [ ]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --source /content/drive/MyDrive/CVFinalproject/004.mp4 --conf-thres=0.75 --save-crop

In [ ]:
!pip install pytesseract
!sudo apt install tesseract-ocr

In [ ]:
import pytesseract
import cv2
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
image = cv2.imread('/content/yolov5/runs/detect/exp4/crops/licence/002345.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.resize(image, (800, 300), interpolation=cv2.INTER_CUBIC)
ret, image = cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, -1, (0, 0, 255), 2)
text = pytesseract.image_to_string(image,
                config = f'--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
#text = pytesseract.image_to_string(image)
cv2_imshow(image)
#plt.imshow(image)
#plt.show()
print(text)

In [ ]:
!pip install easyocr

In [ ]:
import easyocr

In [ ]:
reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory
image = cv2.imread('/content/yolov5/runs/detect/exp4/crops/licence/00236.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.resize(image, (800, 300), interpolation=cv2.INTER_CUBIC)
ret, image = cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#cv2.drawContours(image, contours, -1, (0, 0, 255), 2)
result = reader.readtext(image,detail = 0)
print(result)

In [ ]:
#使用Tesseract

import cv2
import pytesseract
import os
from google.colab.patches import cv2_imshow
from collections import Counter

all_plates = []
# 資料夾路徑
folder_path = '/content/yolov5/runs/detect/exp4/crops/licence/'

# 輸出txt檔案路徑
txt_file_path = '/content/license_plate.txt'

# 初始化txt檔案
with open(txt_file_path, 'w') as file:
    pass

# 讀取資料夾內的所有圖片檔案
image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

for image_file in image_files:
    # 圖片路徑
    image_path = os.path.join(folder_path, image_file)

    # 讀取圖片並進行處理
    image = cv2.imread(image_path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_resized = cv2.resize(image_gray, (800, 300), interpolation=cv2.INTER_CUBIC)
    ret, image_threshold = cv2.threshold(image_resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(image_threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image_threshold, contours, -1, (0, 0, 255), 2)

    # 使用Tesseract進行文字識別
    result = pytesseract.image_to_string(image_threshold,
                                       config='--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    all_plates.append(result)

# 計算車牌出現次數
plate_counts = Counter(all_plates)

# 找到出現最多次的車牌
most_common_plate = plate_counts.most_common(1)[0][0]

# 將結果寫入txt檔案
with open(txt_file_path, 'w') as file:
    file.write(f'Most common license plate: {most_common_plate}\n')

# 輸出結果
print(f'Most common license plate: {most_common_plate}')

In [ ]:
#使用EasyOCR

import cv2
import easyocr
import os
from collections import Counter

# 初始化EasyOCR模型
reader = easyocr.Reader(['en']) # 只需執行一次，將模型載入記憶體

# 資料夾路徑
folder_path = '/content/yolov5/runs/detect/exp15/crops/licence/'

# 輸出txt檔案路徑
txt_file_path = '/content/license_plate.txt'

# 讀取資料夾內的所有圖片檔案
image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

# 儲存所有車牌結果的列表
all_plates = []

for image_file in image_files:
    # 圖片路徑
    image_path = os.path.join(folder_path, image_file)

    # 讀取圖片並進行處理
    image = cv2.imread(image_path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_resized = cv2.resize(image_gray, (800, 300), interpolation=cv2.INTER_CUBIC)
    ret, image_threshold = cv2.threshold(image_resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(image_resized, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image, contours, -1, (0, 0, 255), 2)

    # 使用EasyOCR進行文字識別
    #result = reader.readtext(image_threshold, detail=0)
    #all_plates.extend(result)
    #使用pytesseract
    result = pytesseract.image_to_string(image_threshold,
                              config='--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    print(result)
    # 將車牌識別結果加入列表
    all_plates.append(result)

# 計算車牌出現次數
plate_counts = Counter(all_plates)
print(Counter(all_plates))

# 找到出現最多次的車牌
most_common_plate = plate_counts.most_common(1)[0][0]

# 將結果寫入txt檔案
with open(txt_file_path, 'w') as file:
    file.write(f'Most common license plate: {most_common_plate}\n')

# 輸出結果
print(f'Most common license plate: {most_common_plate}')

In [ ]:
#DEMO

import cv2
import easyocr
import os
from collections import Counter

# 初始化EasyOCR模型
reader = easyocr.Reader(['en']) # 只需執行一次，將模型載入記憶體

# 資料夾路徑
folder_path = '/content/yolov5/runs/detect/exp/crops/licence/'

# 輸出txt檔案路徑
txt_file_path = '/content/license_plate.txt'

# 讀取資料夾內的所有圖片檔案
image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

# 儲存所有車牌結果的列表
all_plates = []

for i in range (6):
    for image_file in image_files:
        # 圖片路徑
        image_path = os.path.join(folder_path, image_file)

        # 讀取圖片並進行處理
        image = cv2.imread(image_path)
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image_resized = cv2.resize(image_gray, (800, 300), interpolation=cv2.INTER_CUBIC)
        ret, image_threshold = cv2.threshold(image_resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        contours, hierarchy = cv2.findContours(image_resized, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(image, contours, -1, (0, 0, 255), 2)


        # 使用EasyOCR進行文字識別
        #result = reader.readtext(image_threshold, detail=0)
        #all_plates.extend(result)


        #使用pytesseract
        result = pytesseract.image_to_string(image_threshold,
                                  config='--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
        #print(result)


        # 將車牌識別結果加入列表
        all_plates.append(result)

    # 計算車牌出現次數
    plate_counts = Counter(all_plates)
    print(Counter(all_plates))

    # 找到出現最多次的車牌
    most_common_plate = plate_counts.most_common(1)[0][0]

    # 將結果寫入txt檔案
    with open(txt_file_path, 'a+') as file:
        file.write(f'Most common license plate: {most_common_plate}')

    # 輸出結果
    print(f'Most common license plate: {most_common_plate}')